## 講習1 --- IRAFを使ってみる

旧来のIRAFの操作は、ターミナルからCLコマンドラインを使って対話的に行うものです。  
この対話的な操作を、python + Jupyter notebookを使っても行うことができます。
ここでは、IRAFの基本的なタスク、**display, imexam, imstat** を使ってみます。  

### pyrafのための準備  

pythonからIRAFを使うにはpyrafを利用します。  
pyrafを(便利に)使うためには、  

- ホームディレクトリにirafというディレクトリを作成
- そのディレクトリ内で mkirafを実行し、login.cl を作成
- 必要に応じてそのlogin.clを編集  
    (今回のサンプルデータの場合、26行目あたりの'#set stdimage = imt800'を  
    'set stdimage = imt4096'にしておくとよいでしょう。冒頭の#を取り除き、800を4096に)

をしておきます。 こうしておくと、login.clでの設定がpyraf利用時に反映されます。また、~/iraf/uparm/に各タスク(imexam, imstatなど)のパラメータが保存されます。　　

**_補足_** :  ~/.iraf/login.cl がある場合には、そちらの設定が使用されます。  
自分のホームディレクトリ化にIRAFをローカルインストールした場合、~/.iraf というディレクトリが作成されます。  

### モジュールの読み込み

In [1]:
from pyraf import iraf 

これでirafのタスクをpythonで関数として使うことができます。(macでは'Warning : sscanf library not ... 'がでることがあります。とりえあず無視してください。)

### サンプルデータ

サンプルデータとして、すばるSuprime Camで取得したデータを使います。 (SMOKAで公開されているデータです。) 同じディレクトリの data1 および data2 の中のFITSファイルです。　　

この講習では、2004-06-21の観測データの一部を利用します。フィルターはBバンドで、ドームフラットおよび二種類のターゲットの生データです。Suprime Camは10枚のCCDから成ります。ファイル名(拡張子除く)の末尾の数字がCCDの番号を示しています。
data1には5番フレーム、data2には2番フレームのデータがあります。例えば、data1/SUPA00317705.fitsとdata2/SUPA00317702.fits は同じ積分のそれぞれ、5番フレームと2番フレームのデータです。講習のなかでは主に5番フレームを用いて説明をします。演習の中で2番フレームを使うことがあります。

| FITSファイル| OBJECT | フィルター | 積分時間(秒) |  
| :- | :- | :- | :- | 
| data1/SUPA003175[0-6]5.fits  |  ドームフラット | B | 10 | 
| data1/SUPA00317705.fits  | target1 | B | 10 | 
| data1/SUPA00317885.fits  | target2 | B | 30 | 
| data2/SUPA003175[0-6]2.fits  |  ドームフラット | B | 10 | 
| data2/SUPA00317702.fits  | target1 | B | 10 | 
| data2/SUPA00317882.fits  | target2 | B | 30 | 

<img src='./img/SUPA0031770X.png', width='500'>  

#### Unixコマンド

Unixコマンドを使い、カレントディレクトリおよびその中身を確認することができます。

In [2]:
pwd

'/Users/nakajima/git/adc2017python2'

In [3]:
ls data1

SUPA00317505.fits  SUPA00317535.fits  SUPA00317565.fits
SUPA00317515.fits  SUPA00317545.fits  SUPA00317705.fits
SUPA00317525.fits  SUPA00317555.fits  SUPA00317885.fits


いくつかのUnixコマンドは、このように、jupyter notebookで直接使えます。使えないものもあります。その場合、! を冒頭につけて使用してやります。

In [6]:
! date

2017年 8月21日 月曜日 10時01分36秒 JST


それでは、IRAFのコマンド、display, imexam, imstatを使って、それらファイルの表示やデータの吟味、統計量の測定をしてみましょう。

### DS9にFITSデータを表示してみる

別のターミナルからds9を立ち上げておきます。  
irafモジュールのdisplay()関数を使い、引数としてファイルを指定します。target1の5番フレームを表示してみます。

In [7]:
iraf.display('data1/SUPA00317705.fits')

z1=10012.29 z2=10105.81


ds9に、星がいくつか写っている視野が表示されましたね。  
(**FAQ** ~/iraf/login.cl で set stdimage = imt800 のままだと視野の中心付近の800x800の領域しか表示されません。)

### imexam 
ds9にFITS画像を表示した状態で、次のコマンドを実行し、マウスカーソルを、星のないところにあてて**m**、あるいは星にあてて**a**や**r**をタイプしてみましょう。終わるときには**q**をタイプしてください。  

**m**は、カウスカーソルを当てた部分の5x5ピクセルの範囲のカウント値の統計を表示します。バックグラウンドの値、ばらつきなどを調べるときに使います。  
**a**は、星(点光源)の特徴量(ピーク値、fwhmなど)を表示します。  
**r**は、星のradial profileを表示します。 星がサチっていれば一目でわかります。  

In [8]:
iraf.imexam()

#            SECTION     NPIX     MEAN   MEDIAN   STDDEV      MIN      MAX
[1170:1174,1963:1967]       25   10060.   10057.    14.95   10041.   10115.
#   COL    LINE     COORDINATES
#     R    MAG    FLUX     SKY    PEAK    E   PA BETA ENCLOSED   MOFFAT DIRECT
1238.67 1913.38 1238.67 1913.38
  21.61  13.08  58358.  10059.    868. 0.04   22 3.00     7.18     7.20   7.20
 21.61  13.08  58358.  10059.    868. 0.04   22 3.00     7.18     7.20   7.20


imexamでは、**r**とか**e**コマンドを使うとさらに別のグラフィックウィンドウが現れます。ブラウザとかの後ろに隠れているかもしれません。

### imstat 
FITS画像の統計量を調べるタスクimstatを使ってみましょう。

In [9]:
iraf.imstat.unlearn()  #  パラメータをデフォルト値に

In [10]:
iraf.imstat('data1/SUPA00317705.fits')

#               IMAGE      NPIX      MEAN    STDDEV       MIN       MAX
 data1/SUPA00317705.fits   8528000    10059.     108.3     9911.    43932.


### eparでパラメータ設定 
**iraf.epar('タスク名')** で、パラメータ設定の画面が別ウィンドウで開きます。GUIでパラメータ設定ができます。
(2017-07 MacOS Sierra + Python3.5.3 + Jupyter Notebook では iraf.epar('display')などでUnicodeDecodeErrorが生じる)

In [11]:
iraf.epar('imstat')


Task imstatistics is running...

#     MIDPT      MEAN    STDDEV
     10058.    10059.     108.3


このように、GUIでもパラメータ設定ができるのですが、ここでは次の方法をおすすめします。  


### 変数としてパラメータ設定

Cellでタスクの変数にパラメータを代入してやります。この方法だと、使ったパラメータがこのノートブックに残るので、あとになって「この処理でどんなパラメータ使ったっけ？」となったときに助けになります。

In [12]:
iraf.imstat.fields = 'midpt, mean, stddev'
iraf.imstat.lower = 9950
iraf.imstat.upper = 11000

In [13]:
iraf.imstat('data1/SUPA00317705.fits')

#     MIDPT      MEAN    STDDEV
     10058.    10058.     15.38


どんな値がパラメータに入っているかを確認するには、それをprint()してやればよいです。

In [14]:
print (iraf.imstat.fields)

midpt, mean, stddev


### 結果の値を変数へ

上ではimstat()の結果が標準出力に表示されました。それぞれの値を変数に保存するには次のようにします。

In [15]:
out = iraf.imstat('data1/SUPA00317705.fits', format='no', Stdout=1) 
# format='no' でヘッダ行非表示、Stdout=1で戻り値を返す

v = out[0].split()  #  戻り値はリスト
median = float(v[0])   # 文字列をfloatに変換しておく
mean = float(v[1])
stddev = float(v[2])
print (median)
print (mean)
print (stddev)
print (median + 3 * stddev) # floatに変換しておかないとここでおかしなことになる

10058.02
10057.7
15.38449
10104.17347


デフォルトではStdout=0なので標準出力に値が返ります。戻り値を変数として返す場合にはStdout=1とします。

### helpドキュメント

このノートブック内でhelpを読むこともできます。  
helpの表示が縦に長すぎる場合、左の余白部分(In[]: の下あたり)をクリックするとスクロールバーつきウインドウ表示になります。
(私の環境でブラウザChromeの場合にはデフォルトでスクロールバー表示されます。)

In [16]:
#iraf.help('imstat')   #  Githubでは表示が長くなってしまうのでコメントアウトしておきます

### 演習1    
data1/SUPA00317885.fitsはtarget2の生データです。  
新しいノートブックファイルを作成し、
1. imexamでバックグラウンドの値とばらつき、星の特徴量、を調べる。
2. imstatでカウント値のmedian, mean, standard deviationを求める。
3. imstatのnclipを2以上にしてみてください。どうなりましたか？